<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-2/Insumos/Merton_KMV_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
import yfinance as yf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
start = '2020-01-01'
symbol = 'WALMEX.MX'
stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


In [3]:
stock_data.info

{'zip': '02770',
 'sector': 'Consumer Defensive',
 'fullTimeEmployees': 231927,
 'longBusinessSummary': "Wal-Mart de México, S.A.B. de C.V. owns and operates self-service stores in Mexico and Central America. The company operates discount warehouses and stores, hypermarkets, supermarkets, and membership self-service wholesale stores. It operates 2,198 Bodega Aurrerá discount stores, 294 Walmart hypermarkets, 14 Superama supermarkets, 85 Walmart Express supermarkets, and 165 Sam's Club membership self-service wholesale stores. The company also operates 572 Despensa Familiar and Palí discount stores; 98 Paiz, La Despensa de Don Juan, La Unión, and Más x Menos supermarkets; 158 Bodegas, Maxi Bodega, and Maxi Palí stores; and 36 Walmart hypermarkets in Costa Rica, Guatemala, Honduras, Nicaragua, and El Salvador. In addition, it imports and sells goods; develops properties; and manages real estate companies. The company was founded in 1958 and is headquartered in Mexico City, Mexico. Wal-Ma

In [4]:
Acc=stock_data.info["sharesOutstanding"]
Kp=stock_data.info["enterpriseValue"]
Dp=stock_data.info["totalDebt"]

In [5]:
Acc=stock_data.info["sharesOutstanding"]
Kp=stock_data.info["enterpriseValue"]
Dp=stock_data.info["totalDebt"]
# print(stock_data.balance_sheet)
import pandas as pd
bs=pd.DataFrame(stock_data.balance_sheet)
K_0=bs[["2021-12-31"]].loc["Total Stockholder Equity"][0]
# bs["Total Stockholder Equity"]
D_0=bs[["2021-12-31"]].loc["Total Liab"][0]

stock_data.balance_sheet


,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Intangible Assets,2.990808e+09,2.554636e+09,1.938229e+09,2.008529e+09
Capital Surplus,4.908573e+09,4.543745e+09,4.318104e+09,4.014804e+09
Total Liab,2.085075e+11,1.927644e+11,1.810275e+11,1.416148e+11
Total Stockholder Equity,1.858820e+11,1.691187e+11,1.681448e+11,1.649141e+11
Other Current Liab,7.001646e+09,6.712921e+09,5.839042e+09,2.738640e+10
Total Assets,3.943895e+11,3.618831e+11,3.491723e+11,3.065288e+11
Common Stock,4.252663e+10,4.252663e+10,4.252663e+10,4.252663e+10
Retained Earnings,1.273109e+11,1.113615e+11,1.088602e+11,1.046387e+11
Other Liab,1.454609e+10,1.317308e+10,1.336026e+10,1.231978e+10
Good Will,3.576724e+10,3.499738e+10,3.514536e+10,3.498921e+10


In [6]:
print(Kp, "vs", K_0)
print(Kp/K_0)
print(Dp, "vs", D_0)
print(Dp/D_0)

1267894190080 vs 185882003000.0
6.820962597869144
68209000448 vs 208507468000.0
0.3271297719082177


In [7]:
import pandas as pd
import numpy as np
from statistics import stdev

df = pd.DataFrame()

hist=stock_data.history(period="max", auto_adjust=True)

df['ds'] = hist.index
df['y'] = hist['Close'].values

df.tail()

,ds,y
5026,2022-11-10 00:00:00-06:00,73.400002
5027,2022-11-11 00:00:00-06:00,72.690002
5028,2022-11-14 00:00:00-06:00,72.120003
5029,2022-11-15 00:00:00-06:00,71.820000
5030,2022-11-16 00:00:00-06:00,72.839996


In [8]:
Delta_A = df[df.ds<="2021-12-31"]['y'].pct_change(1) # 1 for ONE DAY lookback
vol_A=stdev(Delta_A[Delta_A.notnull()])*np.sqrt(252)

vol_A

0.26430736797939924

In [9]:
df[df.ds=="2021-12-31"]['y']*Acc

4808    1.328638e+12
Name: y, dtype: float64

In [10]:
A_0=df[df.ds=="2021-12-31"]['y']
A_0= A_0.values[0]*Acc
print(A_0)


print(A_0/Kp)

print(K_0)
print(A_0/np.array(K_0))
# print(K/A)

r=.05
T=1


1328637827966.125
1.0479090750327456
185882003000.0
7.147748606766008


In [ ]:
from scipy.optimize import fsolve
from scipy.stats import norm
import numpy as np
x=np.array([D_0,vol_A])

K_0=Kp

A_0=A_0*1/2

#Función para estimar vol_k y D
def G(x):
  vol_k = x[1]
  D = x[0]
  F=np.zeros(2)
  F[0]=A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)
  F[1]=A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T))) 
  return F


ss=fsolve(G,x,full_output=True)

print(ss)

# !pip install gekko

# from gekko import GEKKO
# m=GEKKO()
# D,vol_k=[m.Var(1) for i in range(2)]
# m.Equations([A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)==0, A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T))) ==0 ])

# m.solve(disp=False)
# print(D.value,vol_k.value)


print(G(ss[0]))



D=ss[0][0]
vol_k=ss[0][1]

# print(D/D_0)
# print(vol_A/vol_K)


# #K_0=164370

# ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

# D=ss$root[1]
# vol_k=ss$root[2]
# ss$root[1]/D_0
# vol_A/ss$root[2]
# G(c(D,vol_k))
#Probabilidad de incumplimiento
PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
print(PD_merton)

(array([6.34521243e+11, 1.38485053e-01]), {'nfev': 6, 'fjac': array([[-1.00000000e+00,  2.67037404e-07],
       [-2.67037404e-07, -1.00000000e+00]]), 'r': array([-9.51229472e-01, -1.93423340e+05,  1.26789345e+12]), 'qtf': array([ 0.00073242, -0.0038147 ]), 'fvec': array([-1.22070312e-04,  3.05175781e-05])}, 1, 'The solution converged.')
[-1.22070312e-04  3.05175781e-05]
2.8340360835876766e-08


In [ ]:
D=ss[0][0]
vol_k=ss[0][1]

PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
print(PD_merton)

2.8340360835876766e-08


In [ ]:
0.000000028340294555562195

2.8340294555562195e-08

## Conclusiones

1. Verificar el valor de $A_0$, que no sea mucho mayor que $K_0$.
2. Verificar el valor de la deuda.

# KMV


En KMV se encuentra el valor de $K_0$ y $\sigma_K$, dados los demás parámetros.

In [ ]:
print("A_0:", A_0)
print("Valor de K_0", K_0)
print("Valor de vol_A", vol_A)
print("Valor de Deuda Actual", D_0)
print("Tasa libre de riesgo", r)
print("Temporalidad", T)

A_0: 664318913983.0625
Valor de K_0 1267894190080
Valor de vol_A 0.26430737570914636
Valor de Deuda Actual 208507468000.0
Tasa libre de riesgo 0.05
Temporalidad 1


In [ ]:
K_0=Kp

x=np.array([K_0,vol_A])

D=D_0*7
#A_0=A_0*1/2

#Función para estimar vol_k y K_0
def G_kmv(x):
  vol_k = x[1]
  K_0 = x[0]  #modificar el valor inicial de K
  F=np.zeros(2)
  F[0]=A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)
  F[1]=A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T))) 
  return F


ss=fsolve(G_kmv,x,full_output=True)

print(ss)





K_e=ss[0][0]
vol_k=ss[0][1]
print("Cociente K_0/D_0", K_e/D_0)
print("Cociente vol_K/vol_A",vol_k/vol_A)
print("Valor de solución en función:")
G_kmv(ss[0])

DI=(K_e-D)/(vol_k*K_e)
print("DI")
DI
EDF=norm.cdf(-DI,0,1)
print("EDF Normal")
EDF


(array([1.26789419e+12, 2.64307376e-01]), {'nfev': 15, 'fjac': array([[ 0.47671498,  0.87905792],
       [-0.87905792,  0.47671498]]), 'r': array([nan, nan, nan]), 'qtf': array([ 3.06681911e+11, -4.89865112e+11]), 'fvec': array([5.76819668e+11, 3.60651250e+10])}, 5, 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.')
Cociente K_0/D_0 6.080809489662979
Cociente vol_K/vol_A 1.0
Valor de solución en función:
DI
EDF Normal


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  


0.7163117204016772

## Conclusiones

Hace más sentido utilizar KMV cuando $A_0>K_0$, y tiene la ventaja de poder utilizar una deuda "objetivo", para calcular una probabilidad de incumplimiento.